Bidirectional LSTM with Convolution
https://www.kaggle.com/eashish/bidirectional-gru-with-convolution

CountVectorizer, tfdifVectorizer
https://www.kaggle.com/adamschroeder/countvectorizer-tfidfvectorizer-predict-comments?select=test_labels.csv.zip

Improved LSTM baselines
https://www.kaggle.com/jhoward/improved-lstm-baseline-glove-dropout

BERT huggingface transformers
https://www.kaggle.com/nkaenzig/bert-tensorflow-2-huggingface-transformers?select=submission.csv

https://www.kaggle.com/konohayui/bi-gru-cnn-poolings


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

import pickle 
#import mglearn
import time


from nltk.tokenize import TweetTokenizer # doesn't split at apostrophes
import nltk
from nltk import Text
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize  
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

In [2]:
import time
start_time = time.time()
from sklearn.model_selection import train_test_split
import sys, os, re, csv, codecs, numpy as np, pandas as pd
np.random.seed(32)
os.environ["OMP_NUM_THREADS"] = "4"
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
# from keras.engine import InputSpec, Layer

In [3]:
import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [4]:
train = pd.read_csv('train.csv')

embedding_path = "glove.840B.300d.txt"
embed_size = 300
max_features = 100000
max_len = 256

In [5]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [6]:
train.comment_text[12364]

'Media Prostitute is dead. \n\nthis little man, who indiscriminately wiggled his tale in front of those in power, while denigrating his self-credence by repeatedly defending propaganda views with an... authoritative voice of a mere media whore he was, is finally dead. amen.'

In [7]:
train.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [8]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train[list_classes].mean()

toxic            0.095844
severe_toxic     0.009996
obscene          0.052948
threat           0.002996
insult           0.049364
identity_hate    0.008805
dtype: float64

In [14]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf

In [15]:
# list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size = 0.2)

In [16]:
# max_len = 256
# max_features = 100000

raw_text_train = X_train['comment_text'].str.lower()
raw_text_valid = X_valid['comment_text'].str.lower()


tk = Tokenizer(num_words = max_features, lower = True)
tk.fit_on_texts(raw_text_train)
X_train["comment_seq"] = tk.texts_to_sequences(raw_text_train)
X_valid["comment_seq"] = tk.texts_to_sequences(raw_text_valid)


X_train = pad_sequences(X_train.comment_seq, maxlen = max_len)
X_valid = pad_sequences(X_valid.comment_seq, maxlen = max_len)


<ipython-input-16-a559c09586bf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["comment_seq"] = tk.texts_to_sequences(raw_text_train)
<ipython-input-16-a559c09586bf>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid["comment_seq"] = tk.texts_to_sequences(raw_text_valid)


In [17]:
embedding_path = "glove.840B.300d.txt"

def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

In [21]:
X_train[12364]  # len = 256

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [22]:
word_index = tk.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))


for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [23]:
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D

file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(X_valid, y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

In [24]:
def build_model(lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    inp = Input(shape = (max_len,))
    x = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x = SpatialDropout1D(dr)(x)

    x = Bidirectional(GRU(units, return_sequences = True))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])

    x = Dense(6, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    model.summary()
    history = model.fit(X_train, y_train, batch_size = 128, epochs = 4, validation_data = (X_valid, y_valid), 
                        verbose = 1, callbacks = [ra_val, check_point, early_stop])
    model = load_model(file_path)
    return model

In [25]:
model = build_model(lr = 1e-3, lr_d = 0, units = 128, dr = 0.2)

/Users/longmini/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 256, 300)     30000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 256, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 256, 256)     330240      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

In [ ]:
# pred = model.predict(test, batch_size = 1024, verbose = 1)

In [30]:
import tweepy

#Authentication
consumer_key = 'm3rv8Iezfw2wDzxUV6RiqQ3S5'
consumer_secret = '0d3ZMpfbnHTZ5NEJ2xUafERQfIvRyTPfn1zxG1jRHlyJOtgggP'
access_token = '1384998626955464704-vKmpeeRddrjsbUhiUWCTCDk07pZfR1'
access_token_secret = 'jd8sGOI2WwWSWgnSj0h7BAtb3yI7egJcKbVsJnMD1w6pI'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [31]:
numOfItems = 1500
htag = '#palestine'
tweets = []
for tw in tweepy.Cursor(api.search,q = htag).items(numOfItems):
    tweets.append(tw.text)

In [32]:
df = pd.DataFrame(tweets, columns = ['tweets_including_#palestine'])
df

,tweets_including_#palestine
0,RT @sohajml: اذا محكيناش اليوم متى بدنا نحكي ؟...
1,RT @LaraAli63420083: Don't forget Isra.eli how...
2,RT @terrible_stats: 6 ways to lie with statist...
3,#SaveSilwan\n#SaveSheikhJarrah\n#انقذوا_سلوان\...
4,"RT @EELKHAN1: Friend or enemy, they would slau..."
...,...
1495,RT @HudaFadil2: Israeli occupation forces deta...
1496,RT @abierkhatib: “They kicked me in the police...
1497,RT @abierkhatib: “They kicked me in the police...
1498,RT @abierkhatib: Israeli Right wing nationalis...


In [33]:
raw_text_df = df["tweets_including_#palestine"].str.lower()
df["comment_seq"] = tk.texts_to_sequences(raw_text_df)
test_df = pad_sequences(df.comment_seq, maxlen = max_len)

In [34]:
pred_df = model.predict(test_df, batch_size = 1, verbose = 1)

1500/1500 [==============================] - 27s 17ms/step


In [35]:
pred_df

array([[2.2567153e-02, 2.8881133e-03, 2.6811361e-03, 1.2497306e-03,
        2.1367073e-03, 1.4695972e-02],
       [3.7311763e-02, 3.5145879e-04, 8.8873506e-04, 7.4508786e-04,
        2.5208592e-03, 5.4779053e-03],
       [4.8172027e-02, 3.0502677e-04, 1.6862154e-03, 1.2698770e-04,
        1.6115904e-03, 3.4134388e-03],
       ...,
       [2.5685376e-01, 9.1444552e-03, 2.6173919e-02, 2.0071179e-02,
        2.9619604e-02, 3.1123757e-03],
       [3.9883882e-02, 3.4427643e-04, 7.1090460e-04, 4.9746037e-04,
        4.0887892e-03, 1.9652933e-02],
       [2.9409558e-02, 2.2536218e-03, 3.3905804e-03, 2.4625361e-03,
        3.5325885e-03, 5.5477500e-02]], dtype=float32)

In [36]:
df

,tweets_including_#palestine,comment_seq
0,RT @sohajml: اذا محكيناش اليوم متى بدنا نحكي ؟...,"[9501, 3560, 1886]"
1,RT @LaraAli63420083: Don't forget Isra.eli how...,"[9501, 59, 1781, 14008, 70, 6334, 1138, 2299, ..."
2,RT @terrible_stats: 6 ways to lie with statist...,"[9501, 3021, 3563, 546, 1220, 2, 1531, 21, 256..."
3,#SaveSilwan\n#SaveSheikhJarrah\n#انقذوا_سلوان\...,"[3560, 19626]"
4,"RT @EELKHAN1: Friend or enemy, they would slau...","[9501, 971, 25, 3326, 51, 47, 13054, 6, 4, 123..."
...,...,...
1495,RT @HudaFadil2: Israeli occupation forces deta...,"[9501, 1928, 2726, 1953, 72948, 158, 5714, 161..."
1496,RT @abierkhatib: “They kicked me in the police...,"[9501, 28027, 5334, 35, 10, 1, 1284, 2319, 132..."
1497,RT @abierkhatib: “They kicked me in the police...,"[9501, 28027, 5334, 35, 10, 1, 1284, 2319, 132..."
1498,RT @abierkhatib: Israeli Right wing nationalis...,"[9501, 1928, 144, 1989, 5389, 1469, 928, 44, 5..."


In [37]:
test_results = pd.DataFrame(pred_df, columns = list_classes)
test_results

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.022567,0.002888,0.002681,0.001250,0.002137,0.014696
1,0.037312,0.000351,0.000889,0.000745,0.002521,0.005478
2,0.048172,0.000305,0.001686,0.000127,0.001612,0.003413
3,0.029872,0.003893,0.003362,0.002390,0.003557,0.030049
4,0.483323,0.002340,0.005909,0.347408,0.025845,0.003651
...,...,...,...,...,...,...
1495,0.019523,0.001192,0.001341,0.002268,0.001401,0.020523
1496,0.256854,0.009144,0.026174,0.020071,0.029620,0.003112
1497,0.256854,0.009144,0.026174,0.020071,0.029620,0.003112
1498,0.039884,0.000344,0.000711,0.000497,0.004089,0.019653


In [38]:
final = pd.concat([df, test_results], axis = 1)

In [39]:
final

,tweets_including_#palestine,comment_seq,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,RT @sohajml: اذا محكيناش اليوم متى بدنا نحكي ؟...,"[9501, 3560, 1886]",0.022567,0.002888,0.002681,0.001250,0.002137,0.014696
1,RT @LaraAli63420083: Don't forget Isra.eli how...,"[9501, 59, 1781, 14008, 70, 6334, 1138, 2299, ...",0.037312,0.000351,0.000889,0.000745,0.002521,0.005478
2,RT @terrible_stats: 6 ways to lie with statist...,"[9501, 3021, 3563, 546, 1220, 2, 1531, 21, 256...",0.048172,0.000305,0.001686,0.000127,0.001612,0.003413
3,#SaveSilwan\n#SaveSheikhJarrah\n#انقذوا_سلوان\...,"[3560, 19626]",0.029872,0.003893,0.003362,0.002390,0.003557,0.030049
4,"RT @EELKHAN1: Friend or enemy, they would slau...","[9501, 971, 25, 3326, 51, 47, 13054, 6, 4, 123...",0.483323,0.002340,0.005909,0.347408,0.025845,0.003651
...,...,...,...,...,...,...,...,...
1495,RT @HudaFadil2: Israeli occupation forces deta...,"[9501, 1928, 2726, 1953, 72948, 158, 5714, 161...",0.019523,0.001192,0.001341,0.002268,0.001401,0.020523
1496,RT @abierkhatib: “They kicked me in the police...,"[9501, 28027, 5334, 35, 10, 1, 1284, 2319, 132...",0.256854,0.009144,0.026174,0.020071,0.029620,0.003112
1497,RT @abierkhatib: “They kicked me in the police...,"[9501, 28027, 5334, 35, 10, 1, 1284, 2319, 132...",0.256854,0.009144,0.026174,0.020071,0.029620,0.003112
1498,RT @abierkhatib: Israeli Right wing nationalis...,"[9501, 1928, 144, 1989, 5389, 1469, 928, 44, 5...",0.039884,0.000344,0.000711,0.000497,0.004089,0.019653


In [40]:
final[list_classes].mean()

toxic            0.071294
severe_toxic     0.002822
obscene          0.006848
threat           0.015185
insult           0.007223
identity_hate    0.016603
dtype: float32

In [57]:
meandf = pd.DataFrame(final[list_classes].mean(), columns = ['Avg_score'])
meandf

,Avg_score
toxic,0.071294
severe_toxic,0.002822
obscene,0.006848
threat,0.015185
insult,0.007223
identity_hate,0.016603


In [41]:
for column in list_classes:
    print(column, final[final[column] == max(final[column])]['tweets_including_#palestine'])

toxic 77    @amnesty What the fuck you were doing while #I...
Name: tweets_including_#palestine, dtype: object
severe_toxic 572    RT @Pareesa01_: F r e e   Palestine \n\n#FreeP...
Name: tweets_including_#palestine, dtype: object
obscene 77    @amnesty What the fuck you were doing while #I...
Name: tweets_including_#palestine, dtype: object
threat 1413    @WhiteHouse Friend or enemy, they would slaugh...
Name: tweets_including_#palestine, dtype: object
insult 643    @AIPAC Take that #Palestine where it's not eve...
Name: tweets_including_#palestine, dtype: object
identity_hate 219    “You are not jewish.. you are a zionist, zioni...
Name: tweets_including_#palestine, dtype: object


In [47]:
final.iloc[[77, 572, 77, 1413, 643, 219]]

,tweets_including_#palestine,comment_seq,toxic,severe_toxic,obscene,threat,insult,identity_hate
77,@amnesty What the fuck you were doing while #I...,"[14964, 39, 1, 120, 6, 86, 282, 211, 1282, 965...",0.997043,0.238475,0.987301,0.004609,0.432309,0.032824
572,RT @Pareesa01_: F r e e Palestine \n\n#FreeP...,"[9501, 806, 875, 317, 317, 3560]",0.863581,0.374535,0.661646,0.003729,0.094056,0.084994
77,@amnesty What the fuck you were doing while #I...,"[14964, 39, 1, 120, 6, 86, 282, 211, 1282, 965...",0.997043,0.238475,0.987301,0.004609,0.432309,0.032824
1413,"@WhiteHouse Friend or enemy, they would slaugh...","[19451, 971, 25, 3326, 51, 47, 13054, 6, 4, 12...",0.293250,0.004270,0.006119,0.430108,0.022580,0.006028
643,@AIPAC Take that #Palestine where it's not eve...,"[14633, 153, 9, 3560, 129, 77, 14, 96, 265, 2,...",0.854757,0.004998,0.058614,0.001498,0.568008,0.056690
219,"“You are not jewish.. you are a zionist, zioni...","[29467, 19, 14, 911, 6, 19, 5, 4524, 9741, 19,...",0.728612,0.005952,0.034040,0.004050,0.197977,0.565448


In [50]:
print(final['tweets_including_#palestine'][77], "\n")
print(final['tweets_including_#palestine'][572], "\n")
print(final['tweets_including_#palestine'][77], "\n")
print(final['tweets_including_#palestine'][1413], "\n")
print(final['tweets_including_#palestine'][643], "\n")
print(final['tweets_including_#palestine'][219], "\n")

@amnesty What the fuck you were doing while #IsraeliTerrorists Air force was bombing Media towers in #Palestine 

RT @Pareesa01_: F r e e   Palestine 

#FreePalestine #savepalestinians #SavePalestine #savegazachildren #savegazafromzionists #SaveSheikhJa… 

@amnesty What the fuck you were doing while #IsraeliTerrorists Air force was bombing Media towers in #Palestine 

@WhiteHouse Friend or enemy, they would slaughter you and say “it’s your own fault to be in that region”. With this… https://t.co/2dLaUoPqxv 

@AIPAC Take that #Palestine where it's not even fair to be a woman let alone #LGBTQ... Palestine, land of the idiots, home of the fools... 

“You are not jewish.. you are a zionist, zionists are against god” -said by jews 

#freepalastine #SaveSheikhJarrah… https://t.co/Ayqduygj4J 



In [51]:
singletest = ['Fuck you John Doe, you are very arrogant. I will kill John', 'I want her to go to hell, she sucks', 'I hate your FACE']
raw_single = [string.lower() for string in singletest]
seq_single = tk.texts_to_sequences(singletest)
test_single = pad_sequences(seq_single, maxlen = max_len)

In [52]:
raw_single

['fuck you john doe, you are very arrogant. i will kill john',
 'i want her to go to hell, she sucks',
 'i hate your face']

In [53]:
singlepred = model.predict(test_single, batch_size = 1, verbose = 1)

3/3 [==============================] - 0s 26ms/step


In [54]:
singlepred

array([[0.9995462 , 0.5899693 , 0.9857479 , 0.8867403 , 0.90579677,
        0.03410235],
       [0.94786996, 0.07435051, 0.85777354, 0.03405488, 0.3807136 ,
        0.00743109],
       [0.95082784, 0.01936346, 0.09759352, 0.0260607 , 0.1404301 ,
        0.00869071]], dtype=float32)

In [55]:
singledf = pd.DataFrame(singlepred, columns = list_classes)
singledf

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.999546,0.589969,0.985748,0.886740,0.905797,0.034102
1,0.947870,0.074351,0.857774,0.034055,0.380714,0.007431
2,0.950828,0.019363,0.097594,0.026061,0.140430,0.008691
